# Database Playground

This notebook demonstrates the functionalities of our improved Database class.

In [ ]:
import my_functions as mf
import pandas as pd

# Create a Database instance
db = mf.Database()

## 1. Accessing Tables and Views

In [ ]:
# List all tables
print("Available tables:", list(db.tables.keys()))

# Access a specific table
voter_table = db.voter
print("\nVoter table shape:", voter_table().shape)

# List views for a table
print("\nViews in voter table:", voter_table.list_views())

# Access a specific view
voter_2022 = voter_table.get_view('2022')
print("\nVoter 2022 view shape:", voter_2022.shape)
print("\nVoter 2022 view columns:", voter_2022.columns.tolist())

## 2. Adding a New View

In [ ]:
# Add a new view to the voter table
db.add_view('voter', 'registered_voters', ['county_name', 'total_registered_2018', 'total_registered_2020', 'total_registered_2022'])

# Verify the new view
print("Updated views in voter table:", db.voter.list_views())
registered_voters = db.get_view('voter', 'registered_voters')
print("\nRegistered voters view:")
print(registered_voters.head())

## 3. Merging Views

In [ ]:
# Merge multiple views
merged_views = db.merge_views([('demo', 'population'), ('voter', 'registered_voters')])
print("Merged views shape:", merged_views.shape)
print("\nMerged views columns:", merged_views.columns.tolist())
print("\nSample of merged data:")
print(merged_views.head())

## 4. Querying the Database

In [ ]:
# Query for counties with population over 1 million
large_counties = db.query({'population_january_2023': lambda x: x > 1000000}, 
                          ['county_name', 'population_january_2023', 'median_household_income_2021'])
print("Large counties (population > 1 million):")
print(large_counties)

# Query for counties with high median household income
high_income_counties = db.query({'median_household_income_2021': lambda x: x > 100000}, 
                                ['county_name', 'population_january_2023', 'median_household_income_2021'])
print("\nHigh income counties (median household income > $100,000):")
print(high_income_counties)

## 5. Advanced Analysis: Voter Registration Trends

In [ ]:
# Get voter registration data
voter_reg = db.get_view('voter', 'registered_voters')

# Calculate percentage change in voter registration from 2018 to 2022
voter_reg['pct_change_2018_2022'] = (voter_reg['total_registered_2022'] - voter_reg['total_registered_2018']) / voter_reg['total_registered_2018'] * 100

# Sort by percentage change and display top 10 counties with highest increase
top_increase = voter_reg.sort_values('pct_change_2018_2022', ascending=False).head(10)
print("Top 10 counties with highest increase in voter registration (2018-2022):")
print(top_increase[['county_name', 'pct_change_2018_2022']])

# Calculate and display average change across all counties
avg_change = voter_reg['pct_change_2018_2022'].mean()
print(f"\nAverage change in voter registration across all counties: {avg_change:.2f}%")

## 6. Error Handling

In [ ]:
# Try to access a non-existent table
try:
    db.get_view('non_existent_table', 'some_view')
except ValueError as e:
    print("Error:", str(e))

# Try to access a non-existent view
try:
    db.get_view('voter', 'non_existent_view')
except ValueError as e:
    print("Error:", str(e))